# Свёрточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [1]:
# Если Вы запускаете ноутбук на colab или kaggle,
# выполните следующие строчки, чтобы подгрузить библиотеку dlnlputils:

# !git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
# import sys; sys.path.append('./stepik-dl-nlp')

In [2]:
# !pip install pyconll
# !pip install spacy_udpipe

In [35]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, pos_corpus_to_tensor, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed

init_random_seed()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [4]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
# !wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
# !wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
# Windows moment
# import wget

# wget.download('https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu','datasets/ru_syntagrus-ud-train.conllu')
# wget.download('https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu','datasets/ru_syntagrus-ud-dev.conllu')

In [5]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
full_train = pyconll.load_from_file('./datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./datasets/ru_syntagrus-ud-dev.conllu')

In [6]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [7]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [8]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [15]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 142
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [16]:
end_of_word_token_num = max(char_vocab.values()) + 1

char_vocab['<END OF WORD>'] = end_of_word_token_num
char_vocab['<END OF WORD>']

142

In [17]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [19]:
train_inputs, train_labels = pos_corpus_to_tensor(
    full_train,
    char_vocab,
    label2id,
    MAX_SENT_LEN,
    MAX_ORIG_TOKEN_LEN,
    end_of_word_token_num
)
train_dataset = TensorDataset(train_inputs, train_labels)

full_test = full_test[:6587]
test_inputs, test_labels = pos_corpus_to_tensor(
    full_test,
    char_vocab,
    label2id,
    MAX_SENT_LEN,
    MAX_ORIG_TOKEN_LEN,
    end_of_word_token_num
)
test_dataset = TensorDataset(test_inputs, test_labels)

In [20]:
train_inputs[1][:5]

tensor([[142,  38,   4,  25,   4,  11,  19,   7,   6,  13, 142,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,   2,  23,  11,   4,   9,   5,   7,   2,  22,   2, 142,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,  17,  16,  10,   4,  12,  11,   3,   7,   6,  20, 142,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,   9,  12,  20,  21,   6, 142,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0],
        [142,  40,   3,  15,   3,   7, 142,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0]])

In [21]:
train_labels[1]

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

## Вспомогательная свёрточная архитектура

In [22]:
class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for _ in range(layers_n):
            layers.append(nn.Sequential(
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                nn.Dropout(dropout),
                nn.LeakyReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

## Предсказание частей речи на уровне отдельных токенов

In [23]:
class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [24]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 47378


In [25]:
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Эпоха 0
Эпоха: 384 итераций, 43.50 сек
Среднее значение функции потерь на обучении 0.09296515435077406
Среднее значение функции потерь на валидации 0.03290295706010691
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 26.75 сек
Среднее значение функции потерь на обучении 0.02890738768716498
Среднее значение функции потерь на валидации 0.02943419347895254
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 26.77 сек
Среднее значение функции потерь на обучении 0.02463983616811068
Среднее значение функции потерь на валидации 0.025917909578374115
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 26.77 сек
Среднее значение функции потерь на обучении 0.02247584963091261
Среднее значение функции потерь на валидации 0.02298030521868184
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 26.74 сек
Среднее значение функции потерь на обучении 0.02109451394790085
Среднее значение функции потерь на валидации 0.021145942731585245
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 26.75 сек
Среднее з

In [26]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_single_token_model.state_dict(), './models/single_token_pos.pth')

In [27]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
single_token_model.load_state_dict(torch.load('./models/single_token_pos.pth'))

<All keys matched successfully>

In [28]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:12, 63.37it/s]                                                                                               


Среднее значение функции потерь на обучении 0.016877222806215286
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.89      0.94      0.92     43357
         ADP       1.00      0.99      1.00     39344
         ADV       0.92      0.90      0.91     22733
         AUX       0.86      0.72      0.79      3537
       CCONJ       0.88      0.98      0.93     15168
         DET       0.98      0.70      0.82     10781
        INTJ       0.62      0.32      0.42        50
        NOUN       0.97      0.95      0.96    103538
         NUM       0.92      0.93      0.93      5640
        PART       0.96      0.78      0.86     13556
        PRON       0.81      0.95      0.87     18733
       PROPN       0.87      0.92      0.90     14855
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.84      0.86      0.85      8057
         SYM       0.99      0.99      0.99       420
        VERB    

100%|██████████████████████████████████████████████████████████████████████████| 206/205.84375 [00:03<00:00, 60.59it/s]


Среднее значение функции потерь на валидации 0.019597966223955154
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1159365
         ADJ       0.87      0.93      0.90     11535
         ADP       0.99      1.00      0.99     10587
         ADV       0.91      0.89      0.90      6205
         AUX       0.87      0.69      0.77      1109
       CCONJ       0.89      0.98      0.93      4411
         DET       0.98      0.67      0.80      3085
        INTJ       1.00      0.27      0.43        11
        NOUN       0.96      0.94      0.95     27982
         NUM       0.88      0.86      0.87      1474
        PART       0.96      0.79      0.87      3881
        PRON       0.80      0.95      0.87      5600
       PROPN       0.85      0.88      0.87      4438
       PUNCT       1.00      1.00      1.00     22698
       SCONJ       0.83      0.86      0.85      2264
         SYM       1.00      0.87      0.93        53
        VERB   

## Предсказание частей речи на уровне предложений (с учётом контекста)

In [29]:
class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

In [30]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84434


In [31]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=5e-3,
                                              epoch_n=10,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=100,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                         factor=0.5,
                                                                                                                         verbose=True))

Эпоха 0
Эпоха: 384 итераций, 28.04 сек
Среднее значение функции потерь на обучении 0.08227081665730414
Среднее значение функции потерь на валидации 0.029428152791639364
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 27.58 сек
Среднее значение функции потерь на обучении 0.025877638394983176
Среднее значение функции потерь на валидации 0.018920284871122624
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 27.58 сек
Среднее значение функции потерь на обучении 0.020720251832244685
Среднее значение функции потерь на валидации 0.016077696202820776
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 27.58 сек
Среднее значение функции потерь на обучении 0.018676666152411297
Среднее значение функции потерь на валидации 0.015437722132347597
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 27.58 сек
Среднее значение функции потерь на обучении 0.017042507087656606
Среднее значение функции потерь на валидации 0.014431283463456549
Новая лучшая модель!

Эпоха 5
Эпоха: 384 итераций, 27.58 сек
Ср

In [32]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
torch.save(best_sentence_level_model.state_dict(), './models/sentence_level_pos.pth')

In [33]:
# Если Вы запускаете ноутбук на colab или kaggle, добавьте в начало пути ./stepik-dl-nlp
sentence_level_model.load_state_dict(torch.load('./models/sentence_level_pos.pth'))

<All keys matched successfully>

In [34]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:12, 61.77it/s]                                                                                               


Среднее значение функции потерь на обучении 0.00939972698688507
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.94      0.94      0.94     43357
         ADP       1.00      0.99      1.00     39344
         ADV       0.92      0.93      0.92     22733
         AUX       0.86      0.95      0.90      3537
       CCONJ       0.94      0.98      0.96     15168
         DET       0.91      0.93      0.92     10781
        INTJ       1.00      0.22      0.36        50
        NOUN       0.97      0.98      0.97    103538
         NUM       0.96      0.89      0.93      5640
        PART       0.97      0.88      0.92     13556
        PRON       0.95      0.92      0.94     18733
       PROPN       0.96      0.95      0.96     14855
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.84      0.97      0.90      8057
         SYM       0.99      0.99      0.99       420
        VERB     

100%|██████████████████████████████████████████████████████████████████████████| 206/205.84375 [00:03<00:00, 59.25it/s]


Среднее значение функции потерь на валидации 0.012071555480360985
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1159365
         ADJ       0.93      0.92      0.92     11535
         ADP       0.99      0.99      0.99     10587
         ADV       0.91      0.91      0.91      6205
         AUX       0.87      0.95      0.91      1109
       CCONJ       0.95      0.98      0.96      4411
         DET       0.90      0.91      0.91      3085
        INTJ       0.60      0.27      0.37        11
        NOUN       0.96      0.97      0.97     27982
         NUM       0.93      0.85      0.89      1474
        PART       0.97      0.88      0.92      3881
        PRON       0.94      0.91      0.93      5600
       PROPN       0.93      0.92      0.93      4438
       PUNCT       1.00      1.00      1.00     22698
       SCONJ       0.82      0.96      0.89      2264
         SYM       0.96      0.98      0.97        53
        VERB   

## Применение полученных теггеров и сравнение

In [36]:
single_token_pos_tagger = POSTagger(
    single_token_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN, end_of_word_token_num)
sentence_level_pos_tagger = POSTagger(
    sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN, end_of_word_token_num)

In [37]:
test_sentences = [
    'Мама мыла раму.',
    'Косил косой косой косой.',
    'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
    'Сяпала Калуша с Калушатами по напушке.',
    'Пирожки поставлены в печь, мама любит печь.',
    'Ведро дало течь, вода стала течь.',
    'Три да три, будет дырка.',
    'Три да три, будет шесть.',
    'Сорок сорок'
]
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [38]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00,  6.73it/s]                                                                                                 

мама-NOUN мыла-VERB раму-NOUN

косил-VERB косой-ADJ косой-ADJ косой-ADJ

глокая-ADJ куздра-NOUN штеко-ADV будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-NOUN

ведро-NOUN дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-CCONJ три-NUM будет-AUX дырка-NOUN

три-NUM да-CCONJ три-NUM будет-AUX шесть-NUM

сорок-NOUN сорок-NOUN



In [39]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
    print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
    print()

1it [00:00,  9.28it/s]                                                                                                 

мама-NOUN мыла-NOUN раму-NOUN

косил-VERB косой-NOUN косой-NOUN косой-ADJ

глокая-ADJ куздра-NOUN штеко-NOUN будланула-VERB бокра-NOUN и-CCONJ куздрячит-VERB бокрёнка-NOUN

сяпала-VERB калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-VERB в-ADP печь-NOUN мама-NOUN любит-VERB печь-VERB

ведро-ADV дало-VERB течь-NOUN вода-NOUN стала-VERB течь-NOUN

три-NUM да-PART три-NUM будет-AUX дырка-NOUN

три-NUM да-PART три-NUM будет-AUX шесть-VERB

сорок-NOUN сорок-NOUN



## Свёрточный модуль своими руками

In [39]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

In [40]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 84370


In [41]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

Эпоха 0
Эпоха: 384 итераций, 58.94 сек
Среднее значение функции потерь на обучении 0.09000174928708778
Среднее значение функции потерь на валидации 0.021542956683747838
Новая лучшая модель!

Эпоха 1
Эпоха: 384 итераций, 58.26 сек
Среднее значение функции потерь на обучении 0.02319764666511522
Среднее значение функции потерь на валидации 0.017887339788940873
Новая лучшая модель!

Эпоха 2
Эпоха: 384 итераций, 58.38 сек
Среднее значение функции потерь на обучении 0.019892208256351296
Среднее значение функции потерь на валидации 0.015883103939750703
Новая лучшая модель!

Эпоха 3
Эпоха: 384 итераций, 58.53 сек
Среднее значение функции потерь на обучении 0.01845669070947527
Среднее значение функции потерь на валидации 0.01466410596956416
Новая лучшая модель!

Эпоха 4
Эпоха: 384 итераций, 58.41 сек
Среднее значение функции потерь на обучении 0.01701695406518411
Среднее значение функции потерь на валидации 0.015071065888952205

Эпоха 5
Эпоха: 384 итераций, 58.45 сек
Среднее значение функции по

In [42]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

767it [00:22, 34.65it/s]                                                                                               


Среднее значение функции потерь на обучении 0.01015994418412447
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   4330443
         ADJ       0.93      0.94      0.94     43357
         ADP       1.00      0.99      1.00     39344
         ADV       0.92      0.91      0.92     22733
         AUX       0.88      0.94      0.91      3537
       CCONJ       0.93      0.98      0.96     15168
         DET       0.90      0.94      0.92     10781
        INTJ       0.81      0.44      0.57        50
        NOUN       0.98      0.97      0.97    103538
         NUM       0.94      0.96      0.95      5640
        PART       0.97      0.86      0.91     13556
        PRON       0.96      0.91      0.93     18733
       PROPN       0.97      0.94      0.95     14855
       PUNCT       1.00      1.00      1.00     77972
       SCONJ       0.83      0.98      0.90      8057
         SYM       0.99      1.00      0.99       420
        VERB     

100%|██████████████████████████████████████████████████████████████████████████| 206/205.84375 [00:06<00:00, 33.59it/s]


Среднее значение функции потерь на валидации 0.012397450394928455
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1159365
         ADJ       0.92      0.93      0.93     11535
         ADP       0.99      0.99      0.99     10587
         ADV       0.91      0.91      0.91      6205
         AUX       0.87      0.93      0.90      1109
       CCONJ       0.94      0.98      0.96      4411
         DET       0.88      0.91      0.90      3085
        INTJ       0.25      0.36      0.30        11
        NOUN       0.97      0.96      0.97     27982
         NUM       0.91      0.92      0.92      1474
        PART       0.97      0.86      0.91      3881
        PRON       0.95      0.89      0.92      5600
       PROPN       0.95      0.92      0.94      4438
       PUNCT       1.00      1.00      1.00     22698
       SCONJ       0.82      0.98      0.89      2264
         SYM       0.96      0.98      0.97        53
        VERB   